<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Structured Streaming I
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Общие сведения
+ Rate streaming
+ File streaming
+ Kafka streaming

## Общие сведения

Системы поточной обработки данных:
- работают с непрерывным потоком данных
- нужно хранить состояние стрима
- результат обработки быстро появляется в целевой системе
- должны проектироваться с учетом требований к высокой доступности
- важная скорость обработки данных и время зажержки (лаг)

### Примеры систем поточной обработки данных

#### Карточный процессинг
- нельзя терять платежи
- нельзя дублировать платежи
- простой сервиса недопустим
- максимальное время задержки ~ 1 сек
- небольшой поток событий
- OLTP

#### Обработка логов безопасности
- потеря единичных событий допустима
- дублирование единичных событий допустимо
- простой сервиса допустим
- максимальное время задержки ~ 1 час
- большой поток событий
- OLAP

### Виды стриминг систем

#### Real-time streaming
- низкие задержки на обработку
- низкая пропускная способность
- подходят для критичных систем
- пособытийная обработка
- OLTP
- exactly once consistency (нет потери данных и нет дубликатов)

#### Micro batch streaming
- высокие задержки
- высокая пропускная способность
- не подходят для критичных систем
- обработка батчами
- OLAP
- at least once consistency (во время сбоев могут возникать дубликаты)

### Выводы:
+ Существуют два типа систем поточной обработки данных - real-time и micro-batch
+ Spark Structured Streaming является micro-batch системой
+ При работе с большими данными обычно пропускная способность важнее, чем время задержки


## Rate streaming

Самый простой способ создать стрим - использовать `rate` источник. Созданный DF является streaming, о чем нам говорит метод создания `readStream` и атрибут `isStreaming`. `rate` хорошо подходит для тестирования приложений, когда нет возможности подключится к потоку реальных данных

In [2]:
val sdf = spark.readStream.format("rate").load

sdf = [timestamp: timestamp, value: bigint]


[timestamp: timestamp, value: bigint]

In [3]:
sdf.isStreaming

true

У `sdf`, как и у любого DF, есть схема и план выполнения:

In [4]:
sdf.printSchema
sdf.explain(true)

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)

== Parsed Logical Plan ==
StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.RateStreamProvider@47106b8f, rate, [timestamp#4, value#5L]

== Analyzed Logical Plan ==
timestamp: timestamp, value: bigint
StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.RateStreamProvider@47106b8f, rate, [timestamp#4, value#5L]

== Optimized Logical Plan ==
StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.RateStreamProvider@47106b8f, rate, [timestamp#4, value#5L]

== Physical Plan ==
StreamingRelation rate, [timestamp#4, value#5L]


В отличии от обычных DF, у `sdf` нет таких методов, как `show`, `collect`, `take`. Для них также недоступен Dataset API. Поэтому для того, чтобы посмотреть их содержимое, мы должны использовать `console` синк и создать `StreamingQuery`. Процессинг начинается только после вызова метода `start`. `trigger` позволяет настроить, как часто стрим будет читать новые данные и обрабатывать их

In [5]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createConsoleSink(df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("10 seconds"))
    .option("truncate", "false")
    .option("numRows", "20")
}

createConsoleSink: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row]


In [6]:
val sink = createConsoleSink(sdf)

sink = org.apache.spark.sql.streaming.DataStreamWriter@55186f65


org.apache.spark.sql.streaming.DataStreamWriter@55186f65

In [7]:
val sq = sink.start

sq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@65946306


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@65946306

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2020-06-15 19:29:04.976|0    |
|2020-06-15 19:29:05.976|1    |
|2020-06-15 19:29:06.976|2    |
|2020-06-15 19:29:07.976|3    |
|2020-06-15 19:29:08.976|4    |
+-----------------------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2020-06-15 19:29:09.976|5    |
|2020-06-15 19:29:10.976|6    |
|2020-06-15 19:29:11.976|7    |
|2020-06-15 19:29:12.976|8    |
|2020-06-15 19:29:13.976|9    |
|2020-06-15 19:29:14.976|10   |
|2020-06-15 19:29:15.976|11   |
|2020-06-15 19:29:16.976|12

Чтобы остановить DF, можно вызвать метод `stop` к `sdf`, либо получить список всех streming DF и остановить их:

In [10]:
import org.apache.spark.sql.SparkSession

def killAll() = {
    SparkSession
        .active
        .streams
        .active
        .foreach { x =>
                    val desc = x.lastProgress.sources.head.description
                    x.stop
                    println(s"Stopped ${desc}")
        }               
}

killAll: ()Unit


In [23]:
killAll()

Stopped RateStreamV2[rowsPerSecond=1, rampUpTimeSeconds=0, numPartitions=default


lastException: Throwable = null


Создадим стрим, выполняющий запись в `parquet` файл:

In [11]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createParquetSink(df: DataFrame, 
                      fileName: String) = {
    df
    .writeStream
    .format("parquet")
    .option("path", s"datasets/$fileName")
    .option("checkpointLocation", s"chk/$fileName")
    .trigger(Trigger.ProcessingTime("10 seconds"))
}

createParquetSink: (df: org.apache.spark.sql.DataFrame, fileName: String)org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row]


In [24]:
val sink = createParquetSink(sdf, "s1.parquet")

sink = org.apache.spark.sql.streaming.DataStreamWriter@58e88edf


org.apache.spark.sql.streaming.DataStreamWriter@58e88edf

In [25]:
val sq = sink.start

sq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5766e1f5


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5766e1f5

Убедимся, что стрим пишется в файл:

In [30]:
import sys.process._
"ls -alht datasets/s1.parquet".!!

"total 880
drwxr-xr-x    9 t3nq  staff   288B Jun 15 19:40 _spark_metadata
-rw-r--r--    1 t3nq  staff   756B Jun 15 19:40 part-00005-2ea3ca91-ecae-4c9f-8b66-185ffe02ebfe-c000.snappy.parquet
-rw-r--r--    1 t3nq  staff    16B Jun 15 19:40 .part-00005-2ea3ca91-ecae-4c9f-8b66-185ffe02ebfe-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff   756B Jun 15 19:40 part-00002-6e470e5c-34b0-4acd-ac49-3e103bc17c39-c000.snappy.parquet
-rw-r--r--    1 t3nq  staff    16B Jun 15 19:40 .part-00002-6e470e5c-34b0-4acd-ac49-3e103bc17c39-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff   756B Jun 15 19:40 part-00001-e84ae4ad-fb6d-46ea-a5bc-a80262676389-c000.snappy.parquet
-rw-r--r--    1 t3nq  staff    16B Jun 15 19:40 .part-00001-e84ae4ad-fb6d-46ea-a5bc-a8026267638...


Прочитаем файл с помощью Spark:

In [28]:
val rates = spark.read.parquet("datasets/s1.parquet")
println(rates.count)
rates.printSchema
rates.show(5, false)

24
root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)

+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2020-06-15 19:39:05.756|0    |
|2020-06-15 19:39:06.756|1    |
|2020-06-15 19:39:07.756|2    |
|2020-06-15 19:39:08.756|3    |
|2020-06-15 19:39:09.756|4    |
+-----------------------+-----+
only showing top 5 rows



rates = [timestamp: timestamp, value: bigint]


[timestamp: timestamp, value: bigint]

Параллельно внутри одного Spark приложения может работать несколько стримов:

In [32]:
spark.sparkContext.uiWebUrl

Some(http://192.168.88.253:4040)

In [31]:
val consoleSink = createConsoleSink(sdf)
val consoleSq = consoleSink.start

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+



consoleSink = org.apache.spark.sql.streaming.DataStreamWriter@f873134
consoleSq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3eee817b


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3eee817b

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2020-06-15 19:43:24.088|0    |
|2020-06-15 19:43:25.088|1    |
|2020-06-15 19:43:26.088|2    |
|2020-06-15 19:43:27.088|3    |
|2020-06-15 19:43:28.088|4    |
+-----------------------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2020-06-15 19:43:29.088|5    |
|2020-06-15 19:43:30.088|6    |
|2020-06-15 19:43:31.088|7    |
|2020-06-15 19:43:32.088|8    |
|2020-06-15 19:43:33.088|9    |
|2020-06-15 19:43:34.088|10   |
|2020-06-15 19:43:35.088|11   |
|2020-06-15 19:43:36.088|12   |
|2020-06-15 19:43:37.088|13   |
|2020-06-15 19:43:38.088|14   |
+-----------------------+-----+

-------------------------------------------
Batch: 3
---------------

In [33]:
killAll

Stopped RateStreamV2[rowsPerSecond=1, rampUpTimeSeconds=0, numPartitions=default
Stopped RateStreamV2[rowsPerSecond=1, rampUpTimeSeconds=0, numPartitions=default


Напишем функцию, которая добавляет к нашей колонке случайный `ident` аэропорта из датасета [Airport Codes](https://datahub.io/core/airport-codes)  

In [34]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("datasets/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

-RECORD 0------------------------------------------
 ident        | 00A                                
 type         | heliport                           
 name         | Total Rf Heliport                  
 elevation_ft | 11                                 
 continent    | NA                                 
 iso_country  | US                                 
 iso_region   | US-PA                              
 municipality | Bensalem                           
 gps_code     | 00A                 

csvOptions = Map(header -> true, inferSchema -> true)
airports = [ident: string, type: string ... 10 more fields]


[ident: string, type: string ... 10 more fields]

In [36]:
spark.read.text("datasets/airport-codes.csv").show(2, 200, true)

-RECORD 0---------------------------------------------------------------------------------------------------------------------
 value | ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates 
-RECORD 1---------------------------------------------------------------------------------------------------------------------
 value | 00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,,00A,"-74.93360137939453, 40.07080078125"                 
only showing top 2 rows



In [38]:
val idents: Array[String] = airports.select('ident).limit(200).distinct.as[String].collect

idents = Array(00A, 00AA, 00AK, 00AL, 00AR, 00AS, 00AZ, 00CA, 00CL, 00CN, 00CO, 00FA, 00FD, 00FL, 00GA, 00GE, 00HI, 00ID, 00IG, 00II, 00IL, 00IN, 00IS, 00KS, 00KY, 00LA, 00LL, 00LS, 00MD, 00MI, 00MN, 00MO, 00MT, 00N, 00NC, 00NJ, 00NK, 00NY, 00OH, 00OI, 00OK, 00OR, 00PA, 00PN, 00PS, 00S, 00SC, 00SD, 00TA, 00TE, 00TN, 00TS, 00TX, 00UT, 00VA, 00VI, 00W, 00WA, 00WI, 00WN, 00WV, 00WY, 00XS, 01A, 01AK, 01AL, 01AR, 01AZ, 01C, 01CA, 01CL, 01CN, 01CO, 01CT, 01FA, 01FD, 01FL, 01GA, 01GE, 01IA, 01ID, 01II, 01IL, 01IN, 01IS, 01J, 01K, 01KS, 01KY, 01LA, 01LL, 01LS, 01MA, 01MD, 01ME, 01MI, 01MN, 01MO, 01MT, 01NC, 01NE, 01NH, 01NJ, 01NM, 01NV, 01NY, 01OI, 01OK, 01OR, 01PA, 01PN, 01PS, 01SC, 01TA, 01TE, 01TN, 01TS, 01TX, 01U, 01UT, 01VA, 01WA, 01WI, 01WN, 01WT, 01WY, 01XA, 01XS, 02AK, 02...


Array(00A, 00AA, 00AK, 00AL, 00AR, 00AS, 00AZ, 00CA, 00CL, 00CN, 00CO, 00FA, 00FD, 00FL, 00GA, 00GE, 00HI, 00ID, 00IG, 00II, 00IL, 00IN, 00IS, 00KS, 00KY, 00LA, 00LL, 00LS, 00MD, 00MI, 00MN, 00MO, 00MT, 00N, 00NC, 00NJ, 00NK, 00NY, 00OH, 00OI, 00OK, 00OR, 00PA, 00PN, 00PS, 00S, 00SC, 00SD, 00TA, 00TE, 00TN, 00TS, 00TX, 00UT, 00VA, 00VI, 00W, 00WA, 00WI, 00WN, 00WV, 00WY, 00XS, 01A, 01AK, 01AL, 01AR, 01AZ, 01C, 01CA, 01CL, 01CN, 01CO, 01CT, 01FA, 01FD, 01FL, 01GA, 01GE, 01IA, 01ID, 01II, 01IL, 01IN, 01IS, 01J, 01K, 01KS, 01KY, 01LA, 01LL, 01LS, 01MA, 01MD, 01ME, 01MI, 01MN, 01MO, 01MT, 01NC, 01NE, 01NH, 01NJ, 01NM, 01NV, 01NY, 01OI, 01OK, 01OR, 01PA, 01PN, 01PS, 01SC, 01TA, 01TE, 01TN, 01TS, 01TX, 01U, 01UT, 01VA, 01WA, 01WI, 01WN, 01WT, 01WY, 01XA, 01XS, 02AK, 02...

In [39]:
import org.apache.spark.sql.functions._

val litArray = idents.map(x => lit(x))

val someArray = array(litArray:_*)

val result = shuffle(someArray)(0)

// val identSdf = sdf.withColumn("ident", shuffle(array(idents.map(lit(_)):_*))(0))
val identSdf = sdf.withColumn("ident", result)

litArray = Array(00A, 00AA, 00AK, 00AL, 00AR, 00AS, 00AZ, 00CA, 00CL, 00CN, 00CO, 00FA, 00FD, 00FL, 00GA, 00GE, 00HI, 00ID, 00IG, 00II, 00IL, 00IN, 00IS, 00KS, 00KY, 00LA, 00LL, 00LS, 00MD, 00MI, 00MN, 00MO, 00MT, 00N, 00NC, 00NJ, 00NK, 00NY, 00OH, 00OI, 00OK, 00OR, 00PA, 00PN, 00PS, 00S, 00SC, 00SD, 00TA, 00TE, 00TN, 00TS, 00TX, 00UT, 00VA, 00VI, 00W, 00WA, 00WI, 00WN, 00WV, 00WY, 00XS, 01A, 01AK, 01AL, 01AR, 01AZ, 01C, 01CA, 01CL, 01CN, 01CO, 01CT, 01FA, 01FD, 01FL, 01GA, 01GE, 01IA, 01ID, 01II, 01IL, 01IN, 01IS, 01J, 01K, 01KS, 01KY, 01LA, 01LL, 01LS, 01MA, 01MD, 01ME, 01MI, 01MN, 01MO, 01MT, 01NC, 01NE, 01NH, 01NJ, 01NM, 01NV, 01NY, 01OI, 01OK, 01OR, 01PA, 01PN, 01PS, 01SC, 01TA, 01TE, 01TN, 01TS, 01TX, 01U,...


Array(00A, 00AA, 00AK, 00AL, 00AR, 00AS, 00AZ, 00CA, 00CL, 00CN, 00CO, 00FA, 00FD, 00FL, 00GA, 00GE, 00HI, 00ID, 00IG, 00II, 00IL, 00IN, 00IS, 00KS, 00KY, 00LA, 00LL, 00LS, 00MD, 00MI, 00MN, 00MO, 00MT, 00N, 00NC, 00NJ, 00NK, 00NY, 00OH, 00OI, 00OK, 00OR, 00PA, 00PN, 00PS, 00S, 00SC, 00SD, 00TA, 00TE, 00TN, 00TS, 00TX, 00UT, 00VA, 00VI, 00W, 00WA, 00WI, 00WN, 00WV, 00WY, 00XS, 01A, 01AK, 01AL, 01AR, 01AZ, 01C, 01CA, 01CL, 01CN, 01CO, 01CT, 01FA, 01FD, 01FL, 01GA, 01GE, 01IA, 01ID, 01II, 01IL, 01IN, 01IS, 01J, 01K, 01KS, 01KY, 01LA, 01LL, 01LS, 01MA, 01MD, 01ME, 01MI, 01MN, 01MO, 01MT, 01NC, 01NE, 01NH, 01NJ, 01NM, 01NV, 01NY, 01OI, 01OK, 01OR, 01PA, 01PN, 01PS, 01SC, 01TA, 01TE, 01TN, 01TS, 01TX, 01U,...

In [50]:
val identPqSink = createParquetSink(identSdf, "s2.parquet")
val identPqSq = identPqSink.start

identPqSink = org.apache.spark.sql.streaming.DataStreamWriter@4fc975c2
identPqSq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@61525207


lastException: Throwable = null


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@61525207

Проверим, что данные записываются в `parquet`

In [42]:
val identPq = spark.read.parquet("datasets/s2.parquet")
println(identPq.count)
identPq.printSchema
identPq.show(5, false)

346687
root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
 |-- ident: string (nullable = true)

+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2020-06-07 20:36:21.019|269  |03FA |
|2020-06-07 20:36:37.019|285  |01WY |
|2020-06-07 20:36:53.019|301  |00IS |
|2020-06-07 20:37:09.019|317  |02ID |
|2020-06-07 20:37:25.019|333  |02AZ |
+-----------------------+-----+-----+
only showing top 5 rows



identPq = [timestamp: timestamp, value: bigint ... 1 more field]


[timestamp: timestamp, value: bigint ... 1 more field]

Временно остановим стрим, он понадобится нам для следующих экспериментов

In [43]:
killAll

Stopped RateStreamV2[rowsPerSecond=1, rampUpTimeSeconds=0, numPartitions=default


### Выводы:
- `rate` - самый простой способ создать стрим для тестирования приложений
- стрим начинает работу после вызова метода `start` и не блокирует основной поток программы
- в одном Spark приложении может работать несколько стримов одновременно

## File Streaming
Spark позволяет запустить стрим, который будет "слушать" директорию и читать из нее новые файлы. При этом за раз будет прочитано количество файлов, установленное в параметре `maxFilesPerTrigger` [ссылка](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#input-sources). В этом кроется одна из основных проблем данного источника. Поскольку стрим, сконфигурированный под чтение небольших файлов, может "упасть", если в директорию начнут попадать файлы большого объема. Создадим стрим из директории `datasets/s2.parquet`:

In [44]:
val sdfFromParquet = spark
        .readStream
        .format("parquet")
        .option("maxFilesPerTrigger", "1")
        .option("path", "datasets/s2.parquet")
        .load

sdfFromParquet.printSchema

Name: java.lang.IllegalArgumentException
Message: Schema must be specified when creating a streaming source DataFrame. If some files already exist in the directory, then depending on the file format you may be able to create a static DataFrame on that directory with 'spark.read.load(directory)' and infer schema from it.
StackTrace:   at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:233)
  at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:95)
  at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:95)
  at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:33)
  at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:215)

In [51]:
"ls -alh datasets/s2.parquet".!!

"total 1200
drwxr-xr-x  153 t3nq  staff   4.8K Jun 15 20:13 .
drwxr-xr-x   18 t3nq  staff   576B Jun 15 20:12 ..
-rw-r--r--    1 t3nq  staff    16B Jun 15 20:13 .part-00000-3f4ce802-057c-4841-a619-ebdb94d67e8a-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff    16B Jun 15 20:13 .part-00000-4a690777-b07a-4e9d-b032-083ac29282f9-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff    16B Jun 15 20:13 .part-00000-5a69533e-65bb-4599-b44d-99d8e205f9fe-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff    16B Jun 15 20:13 .part-00000-9c287510-1f37-4abd-a5ee-dc4a48f4b1dc-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff    16B Jun 15 20:13 .part-00000-a9edd022-ef9a-4c6d-863b-833ab82a0dca-c000.snappy.parquet.crc
-rw-r--r--    1 t3nq  staff    16B Jun 15 20:12 .part-00000-bf...


In [52]:
killAll

Stopped RateStreamV2[rowsPerSecond=1, rampUpTimeSeconds=0, numPartitions=default


Поскольку в директорию могут попасть любые данные, а df должен иметь фиксированную схему, то Spark не позволяет нам создавать SDF на основе файлов без указания схемы.

In [45]:
import org.apache.spark.sql.functions._

val letters = List("a", "b", "c", "d", "e", "f", "g", "i")
val condition = letters.map { x => col("ident").startsWith(x) }.reduce { (x,y) => x or y }

val sdfFromParquet = spark
        .readStream
        .format("parquet")
        .schema(identPq.schema)
        .option("maxFilesPerTrigger", "10")
        .option("path", "datasets/s2.parquet")
        .load
        .withColumn("ident", lower('ident))

sdfFromParquet.printSchema

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
 |-- ident: string (nullable = true)



letters = List(a, b, c, d, e, f, g, i)
condition = (((((((startswith(ident, a) OR startswith(ident, b)) OR startswith(ident, c)) OR startswith(ident, d)) OR startswith(ident, e)) OR startswith(ident, f)) OR startswith(ident, g)) OR startswith(ident, i))
sdfFromParquet = [timestamp: timestamp, value: bigint ... 1 more field]


lastException: Throwable = null


[timestamp: timestamp, value: bigint ... 1 more field]

In [57]:
val consoleSink = createConsoleSink(sdfFromParquet)
val sq = consoleSink.start

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2020-06-15 20:12:35.407|0    |02ps |
|2020-06-15 20:12:36.407|1    |00ts |
|2020-06-15 20:12:37.407|2    |00wa |
|2020-06-15 20:12:38.407|3    |00il |
|2020-06-15 20:12:40.407|5    |02ne |
|2020-06-15 20:12:41.407|6    |01ks |
|2020-06-15 20:12:42.407|7    |02ge |
|2020-06-15 20:12:43.407|8    |00or |
|2020-06-15 20:12:39.407|4    |01k  |
+-----------------------+-----+-----+



consoleSink = org.apache.spark.sql.streaming.DataStreamWriter@7b850df9
sq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2e750a7


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2e750a7

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2020-06-15 20:12:44.407|9    |01nv |
|2020-06-15 20:12:45.407|10   |00nj |
|2020-06-15 20:12:46.407|11   |00wy |
|2020-06-15 20:12:47.407|12   |00ak |
|2020-06-15 20:12:48.407|13   |02wi |
|2020-06-15 20:12:49.407|14   |00sd |
|2020-06-15 20:12:50.407|15   |01ls |
|2020-06-15 20:12:51.407|16   |02ny |
|2020-06-15 20:12:52.407|17   |02or |
|2020-06-15 20:12:53.407|18   |00cn |
+-----------------------+-----+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2020-06-15 20:12:54.407|19   |03ak |
|2020-06-15 20:12:55.407|20   |02ia |
|2020-06-15 20:12:56.407|21   |00or |
|2020-06-15 20:12:57.407|22   |02nj |
|2020-0

In [56]:
killAll

Stopped FileStreamSource[file:/Users/t3nq/Projects/smz/de-spark-scala/datasets/s2.parquet]


File source позволяет со всеми типами файлов, с которыми умеет работать Spark: `parquet`, `orc`, `csv`, `json`, `text`.

In [59]:
while(sq.status.isTriggerActive) {
    Unit
}

sq.stop



In [60]:
spark.streams.active

Array()

### Выводы:
- Spark позволяет создавать SDF на базе всех поддерживаемых типов файлов
- При создании SDF вы должны указать схему данных
- File streaming имеет несколько серьезных недостатков:
  + Входной поток можно ограничить только макисмальным количество файлов, попадающих в батч
  + Если стрим упадает посередине файла, то при перезапуске эти данные будут обработаны еще раз

<img align="right" width="100" height="100" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Apache_kafka.svg/1200px-Apache_kafka.svg.png">

## Kafka streaming

https://kafka.apache.org

**Apache Kafka** - самая распространенная в мире система, на основе которой строятся приложения для поточной обработки данных. Она имеет несколько преимуществ:
- высокая пропускная способность
- высокая доступность за счет распределенной архитектуры и репликации
- у каждого сообщения есть свой номер, который называется offset, что позволяет гранулярно сохранять состояние стрима

### Архитектура системы

#### Topic
Топик - это таблицы в Kafka. Мы пишем данные в топик и читаем данные из топика. Топик как правило распределен по нескольким узлам кластера для обеспечения высокой доступности и скорости работы с данными

<img align="center" width="500" height="500" src="https://kafka.apache.org/25/images/log_anatomy.png">

#### Partition
Партиции - это блоки, из которых состоят топики. Партиция представляет собой неделимый блок, который хранится на одном из узлов. Топик может иметь произвольное количество партиций. Чем больше партиций - тем выше параллелзим при чтении и записи, однако слишком большое число партиций в топике может привести к замедлению работы всей системы.

#### Replica
Каждая партиция имеет (может иметь) несколько реплик. Внешние приложения всегда работают (читают и пишут) с основной репликой. Остальные реплики являются дочерними и не используются во внешнем IO. Если узел, на котором расположена основная реплика, падает, то одна из дочерних реплик становится основной и работа с данными продолжается

#### Message
Сообщения - это данные, которые мы пишем и читаем в Kafka. Они представлены кортежем (Key, Value), но ключ может быть иметь значение `null` (используется не всегда). Сереализация и десереализация данных всегда происходит на уровне клиентов Kafka. Сама Kafka ничего о типах данных не знает и хранит ключи и значения в виде массива байт

#### Offset
Оффсет - это порядковый номер сообщения в партиции. Когда мы пишем сообщение (сообщение всегда пишется в одну из партиций топика), Kafka помещает его в топик с номер `n+1`, где `n` - номер последнего сообщения в этом топике

<img align="center" width="400" height="400" src="https://kafka.apache.org/25/images/log_consumer.png">

#### Producer
Producer - это приложение, которое пишет в топик. Producer'ов может быть много. Параллельная запись достигается за счет того, что каждое новое сообщение попадает в случайную партицию топика (если не указан `key`)

#### Consumer
Consumer - это приложение, читающее данные из топика. Consumer'ов может быть много, в этом случае они называются `consumer group`. Параллельное чтение достигается за счет распределения партиций топика между consumer'ами в рамках одной группы. Каждый consumer читает данные из "своих" партиций и ничего про другие не знает. Если consumer падает, то "его" партиции переходят другим consumer'ам.

#### Commit
Коммитом в Kafka называют сохранение информации о факте обработки сообщения с определенным оффсетом. Поскольку оффсеты для каждой партиции топика свои, то и информация о последнем обработанном оффсете хранится по каждой партиции отдельно. Обычные приложения пишут коммиты в специальный топик Kafka, который имеет название `__consumer_offsets`. Spark хранит обработанные оффсеты по каждому батчу в ФС (например, в HDFS).

#### Retention
Поскольку кластер Kafka не может хранить данные вечно, то в ее конфигурации задаются пороговые значение по **объему** и **времени хранения** для каждого топика, при превышении которых данные удаляются. Например, если у топика A установлен renention по времени 1 месяц, то данные будут хранится в системе не менее одного месяца (и затем будут удалены одной из внутренних подсистем)

### Spark connector
https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10  
https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html  

### Запуск Kafka в docker
```shell
docker run --rm \
   -p 2181:2181 \
   --name=test_zoo \
   -e ZOOKEEPER_CLIENT_PORT=2181 \
   confluentinc/cp-zookeeper
```

```shell
docker run --rm \
    -p 9092:9092 \
    --name=test_kafka \
    -e KAFKA_ZOOKEEPER_CONNECT=host.docker.internal:2181 \
    -e KAFKA_ADVERTISED_LISTENERS=PLAINTEXT://localhost:9092 \
    -e KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR=1 \
    confluentinc/cp-kafka
```

### Работа с Kafka с помощь Static Dataframe

Spark позволяет работать с кафкой как с обычной базой данных. Запишем данные в топик `test_topic0`. Для этого нам необходимо подготовить DF, в котором будет две колонки:
- `value: String` - данные, которые мы хотим записать
- `topic: String` - топик, куда писать каждую строку DF

In [61]:
val identPq = spark.read.parquet("datasets/s2.parquet").limit(200)
println(identPq.count)
identPq.printSchema
identPq.show(5, false)

94
root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
 |-- ident: string (nullable = true)

+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2020-06-15 20:12:35.407|0    |02PS |
|2020-06-15 20:12:36.407|1    |00TS |
|2020-06-15 20:12:37.407|2    |00WA |
|2020-06-15 20:12:38.407|3    |00IL |
|2020-06-15 20:12:40.407|5    |02NE |
+-----------------------+-----+-----+
only showing top 5 rows



identPq = [timestamp: timestamp, value: bigint ... 1 more field]


[timestamp: timestamp, value: bigint ... 1 more field]

In [62]:
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

def writeKafka[T](topic: String, data: Dataset[T]): Unit = {
    val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "localhost:9092"
    )
    
    data.toJSON.withColumn("topic", lit(topic)).write.format("kafka").options(kafkaParams).save
}

writeKafka: [T](topic: String, data: org.apache.spark.sql.Dataset[T])Unit


In [63]:
writeKafka("test_topic0", identPq)

Прочитаем данные из Kafka:

In [71]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "localhost:9092",
        "subscribePattern" -> ".*"
    )


val df = spark.read.format("kafka").options(kafkaParams).load

df.printSchema
df.show

df.groupBy('topic, 'partition).count.show(20, false)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 74 69 6D 6...|test_topic0|        0|     0|2020-06-15 21:15:...|            0|
|null|[7B 22 74 69 6D 6...|test_topic0|        0|     1|2020-06-15 21:15:...|            0|
|null|[7B 22 74 69 6D 6...|test_topic0|        0|     2|2020-06-15 21:15:...|            0|
|null|[7B 22 74 69 6D 6...|test_topic0|        0|     3|2020-06-15 21:15:...|            0|
|null|[7B 22 74 69 6D 6...|test_topic0|        0|     4|2020-06-15 21:15:

kafkaParams = Map(kafka.bootstrap.servers -> localhost:9092, subscribePattern -> .*)
df = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

Чтение из Kafka имеет несколько особенностей:
- по умолчанию читается все содержимое топика. Поскольку обычно в нем много данных, эта операция может создать большую нагрузку на кластер Kafka и Spark приложение
- колонки `value` и `key` имеют тип `binary`, который необходимо десереализовать

Чтобы прочитать только определенную часть топика, нам необходимо задать минимальный и максимальный оффсет для чтения с помощью параметров `startingOffsets` , `endingOffsets`. Возьмем два случайных события:

In [65]:
df.sample(0.1).limit(2).select('topic, 'partition, 'offset).show

+-----------+---------+------+
|      topic|partition|offset|
+-----------+---------+------+
|test_topic0|        0|     9|
|test_topic0|        0|    11|
+-----------+---------+------+



На основании этих событий подготовим параметры `startingOffsets` и `endingOffsets`

In [66]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "localhost:9092",
        "subscribe" -> "test_topic0",
        "startingOffsets" -> """ { "test_topic0": { "0": 9 } } """,
        "endingOffsets" -> """ { "test_topic0": { "0": 11 } }  """
    )


val df = spark.read.format("kafka").options(kafkaParams).load

df.printSchema
df.show(20)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 74 69 6D 6...|test_topic0|        0|     9|2020-06-15 21:15:...|            0|
|null|[7B 22 74 69 6D 6...|test_topic0|        0|    10|2020-06-15 21:15:...|            0|
+----+--------------------+-----------+---------+------+--------------------+-------------+



kafkaParams = Map(kafka.bootstrap.servers -> localhost:9092, subscribe -> test_topic0, startingOffsets -> " { "test_topic0": { "0": 9 } } ", endingOffsets -> " { "test_topic0": { "0": 11 } }  ")
df = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

По умолчанию параметр `startingOffsets` имеет значение `earliest`, а `endingOffsets` - `latest`. Поэтому, когда мы не указывали эти параметры, Spark прочитал содержимое всего топика

Чтобы получить наши данные, которые мы записали в топик, нам необходимо их десереализовать. В нашем случае достаточно использовать `.cast("string")`, однако это работает не всегда, т.к. формат данных может быть произвольным.

In [67]:
val jsonString = df.select('value.cast("string")).as[String]

jsonString.show(20, false)

val parsed = spark.read.json(jsonString)
parsed.printSchema
parsed.show(20, false)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|{"timestamp":"2020-06-15T20:12:45.407+03:00","value":10,"ident":"00NJ"}|
|{"timestamp":"2020-06-15T20:12:46.407+03:00","value":11,"ident":"00WY"}|
+-----------------------------------------------------------------------+

root
 |-- ident: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- value: long (nullable = true)

+-----+-----------------------------+-----+
|ident|timestamp                    |value|
+-----+-----------------------------+-----+
|00NJ |2020-06-15T20:12:45.407+03:00|10   |
|00WY |2020-06-15T20:12:46.407+03:00|11   |
+-----+-----------------------------+-----+



jsonString = [value: string]
parsed = [ident: string, timestamp: string ... 1 more field]


[ident: string, timestamp: string ... 1 more field]

### Работа с Kafka с помощью Streaming DF
При создании SDF из Kafka необходимо помнить, что:
- `startingOffsets` по умолчанию имеет значение `latest`
- `endingOffsets` использовать нельзя
- количество сообщений за батч можно (и нужно) ограничить параметром `maxOffsetPerTrigger` (по умолчанию он не задан и первый батч будет содержать данные всего топика

In [74]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "localhost:9092",
        "subscribe" -> "test_topic0",
        "startingOffsets" -> """earliest""",
        "maxOffsetsPerTrigger" -> "5"
    )

val sdf = spark.readStream.format("kafka").options(kafkaParams).load
val parsedSdf = sdf.select('value.cast("string"), 'topic, 'partition, 'offset)

val sink = createConsoleSink(parsedSdf)

val sq = sink.start

kafkaParams = Map(kafka.bootstrap.servers -> localhost:9092, subscribe -> test_topic0, startingOffsets -> earliest, maxOffsetsPerTrigger -> 5)
sdf = [key: binary, value: binary ... 5 more fields]
parsedSdf = [value: string, topic: string ... 2 more fields]
sink = org.apache.spark.sql.streaming.DataStreamWriter@1ead7eec
sq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3f8e8a17


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3f8e8a17

-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------------------------------------------+-----------+---------+------+
|value                                                                 |topic      |partition|offset|
+----------------------------------------------------------------------+-----------+---------+------+
|{"timestamp":"2020-06-15T20:12:35.407+03:00","value":0,"ident":"02PS"}|test_topic0|0        |0     |
|{"timestamp":"2020-06-15T20:12:36.407+03:00","value":1,"ident":"00TS"}|test_topic0|0        |1     |
|{"timestamp":"2020-06-15T20:12:37.407+03:00","value":2,"ident":"00WA"}|test_topic0|0        |2     |
|{"timestamp":"2020-06-15T20:12:38.407+03:00","value":3,"ident":"00IL"}|test_topic0|0        |3     |
|{"timestamp":"2020-06-15T20:12:40.407+03:00","value":5,"ident":"02NE"}|test_topic0|0        |4     |
+----------------------------------------------------------------------+-----------+---

In [75]:
killAll

Stopped KafkaV2[Subscribe[test_topic0]]


Если мы перезапустим этот стрим, он повторно прочитает все данные. Чтобы обеспечить сохранение состояния стрима после обработки каждого батча, нам необходимо добавить параметр `checkpointLocation` в опции `writeStream`:

In [76]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createConsoleSinkWithCheckpoint(chkName: String, df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("10 seconds"))
    .option("checkpointLocation", s"chk/$chkName")
    .option("truncate", "false")
    .option("numRows", "20")
}

createConsoleSinkWithCheckpoint: (chkName: String, df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row]


In [83]:
"cat chk/test0/offsets/5".!!

"v1
{"batchWatermarkMs":0,"batchTimestampMs":1592246100006,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"200"}}
{"test_topic0":{"0":30}}
"


In [79]:
val sink = createConsoleSinkWithCheckpoint("test0", parsedSdf)
val sq = sink.start

sink = org.apache.spark.sql.streaming.DataStreamWriter@254ca66c
sq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@333fee21


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@333fee21

-------------------------------------------
Batch: 3
-------------------------------------------
+-----------------------------------------------------------------------+-----------+---------+------+
|value                                                                  |topic      |partition|offset|
+-----------------------------------------------------------------------+-----------+---------+------+
|{"timestamp":"2020-06-15T20:12:51.407+03:00","value":16,"ident":"02NY"}|test_topic0|0        |15    |
|{"timestamp":"2020-06-15T20:12:52.407+03:00","value":17,"ident":"02OR"}|test_topic0|0        |16    |
|{"timestamp":"2020-06-15T20:12:53.407+03:00","value":18,"ident":"00CN"}|test_topic0|0        |17    |
|{"timestamp":"2020-06-15T20:12:54.407+03:00","value":19,"ident":"03AK"}|test_topic0|0        |18    |
|{"timestamp":"2020-06-15T20:12:55.407+03:00","value":20,"ident":"02IA"}|test_topic0|0        |19    |
+-----------------------------------------------------------------------+------

In [104]:
parsedSdf.writeStream.foreachBatch { (batchDf, batchId) => println(batchDf.isStreaming) }.start

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6b9f5188

false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false


In [105]:
killAll

Stopped KafkaV2[Subscribe[test_topic0]]


### Выводы:
- Apache Kafka - распределенная система, обеспечивающая передачу потока данных в слабосвязанных системах
- Работать с Kafka можно как с использованием Static DF, так и с помощью Streaming DF
- Чтобы стрим запоминал свое состояние после остановки, необходимо использовать checkpoint - директорию на HDFS (или локальной ФС), в которую будет сохранятся состояние стрима после каждого батча

В конце работы не забудьте остановить Spark:

In [84]:
val localVec = Vector("""{"timestamp":"2020-06-15T20:12:56.407+03:00","value":21,"ident":"00OR"}""")

localVec = Vector({"timestamp":"2020-06-15T20:12:56.407+03:00","value":21,"ident":"00OR"})


Vector({"timestamp":"2020-06-15T20:12:56.407+03:00","value":21,"ident":"00OR"})

In [85]:
val df = localVec.toDF

df = [value: string]


[value: string]

In [101]:
// get_json_object
// json_tuple
// from_json
import org.apache.spark.sql.functions._
// df.select(get_json_object('value, "$.value").alias("foo")).show
// df.select(json_tuple('value, "timestamp", "value", "ident").as(Array("timestamp", "value", "ident"))).show

import org.apache.spark.sql.types._

val docType = StructType(
    StructField("value", StringType) ::
    StructField("timestamp", TimestampType) ::
    StructField("ident", StringType) :: 
    StructField("foo", ArrayType(StructType(
        StructField("bar", StringType) :: Nil
    ))) :: Nil
)

df.select(from_json('value, docType).alias("foo")).select(col("foo.*")).select(col("foo")(0)("bar")).printSchema

root
 |-- foo[0].bar: string (nullable = true)



docType = StructType(StructField(value,StringType,true), StructField(timestamp,TimestampType,true), StructField(ident,StringType,true), StructField(foo,ArrayType(StructType(StructField(bar,StringType,true)),true),true))


StructType(StructField(value,StringType,true), StructField(timestamp,TimestampType,true), StructField(ident,StringType,true), StructField(foo,ArrayType(StructType(StructField(bar,StringType,true)),true),true))

In [ ]:
spark.stop